In [1]:
from pre_process.load_sleep_data import LoadSleepData
from pre_process.pre_process import PreProcess
import numpy as np
import sys, os
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
import sys
from sklearn.manifold import TSNE
from collections import Counter
import tensorflow as tf
from nn.losses import EDLLoss

In [2]:
os.getcwd()

'c:\\Users\\taiki\\git\\sleep_study'

In [4]:
load_sleep_data = LoadSleepData(data_type="spectrum"")
pre_process = PreProcess(load_sleep_data)
datas = load_sleep_data.load_data(load_all=True)
(train, test) = pre_process.split_train_test_from_records(datas,
                                                          test_id=0,
                                                          pse_data=False)
(x_train, y_train), (_, _) = pre_process.make_dataset(train=train, 
                                                      test=test, 
                                                      is_storchastic=False,
                                                      to_one_hot_vector=False,
                                                      pse_data=False)

*** すべての被験者を読み込みます（load_dataの引数:nameは無視します） ***
D:\takadamalab\sleep_study\datas\pre_processed_data\H_Li_20210201-055748.savを読み込んでいます...
D:\takadamalab\sleep_study\datas\pre_processed_data\H_Murakami_20210201-055748.savを読み込んでいます...
D:\takadamalab\sleep_study\datas\pre_processed_data\H_Yamamoto_20210201-055748.savを読み込んでいます...
D:\takadamalab\sleep_study\datas\pre_processed_data\H_Kumazawa_20210201-055748.savを読み込んでいます...
D:\takadamalab\sleep_study\datas\pre_processed_data\H_Hayashi_20210201-055748.savを読み込んでいます...
D:\takadamalab\sleep_study\datas\pre_processed_data\H_Kumazawa_F_20210201-055748.savを読み込んでいます...
D:\takadamalab\sleep_study\datas\pre_processed_data\H_Takadama_20210201-055748.savを読み込んでいます...
D:\takadamalab\sleep_study\datas\pre_processed_data\H_Hiromoto_20210201-055748.savを読み込んでいます...
D:\takadamalab\sleep_study\datas\pre_processed_data\H_Kashiwazaki_20210201-055748.savを読み込んでいます...
訓練データのサイズを揃えます
訓練データの各睡眠段階（補正前） Counter({2: 3340, 4: 1184, 5: 499, 3: 395, 1: 181, None: 9})
訓練データの

In [5]:
# モデルを25次元の空間に出力
date_id = "20210601-050450"
path = os.path.join(os.environ["sleep"], "models", "H_Li", date_id)
model = tf.keras.models.load_model(path, custom_objects={"EDLLoss":EDLLoss(K=5,annealing=0.1)})

ValueError: Unable to restore custom object of type _tf_keras_metric currently. Please make sure that the layer implements `get_config`and `from_config` when saving. In addition, please use the `custom_objects` arg when calling `load_model()`.

In [3]:
# pcaの実装
def make_pca(data=None, target=None, pse_data=True, path=None, show_data=True,
             test_id=0):
    # 仮データを指定したときは読み込む
    if pse_data:
        assert data==None
        digits = datasets.load_digits()
        data = digits.data
        target = digits.target
    # pcaを実行
    X_reduced = PCA(n_components=2).fit_transform(data)
    # データを表示する際はデータを返さない
    if show_data:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        mappable = ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=target)
        plt.title("pca", fontsize=20)
        fig.colorbar(mappable, ax=ax)
        plt.show()
        plt.clf()
    else:
        return X_reduced, target
make_pca(data=x_train, target=y_train, show_data=False, pse_data=False)